In [2]:
import pandas as pd

from us_election_2024.helpers import read_data, unique, filter_for, total_votes_per_precinct, filter_isin, contains

In [3]:
df = read_data()
df

,County,Election Date,Precinct,Contest Group ID,Contest Type,Contest Name,Choice,Choice Party,Vote For,Election Day,Early Voting,Absentee by Mail,Provisional,Total Votes,Real Precinct,Unnamed: 15
0,CALDWELL,11/05/2024,1,1155,S,NC DISTRICT COURT JUDGE DISTRICT 36 SEAT 09,Richard S. Holloway,REP,1,580,0,0,0,580,Y,NaN
1,CALDWELL,11/05/2024,1,1347,S,NC STATE SENATE DISTRICT 45,Kim Bost,DEM,1,173,0,0,0,173,Y,NaN
2,CALDWELL,11/05/2024,1,1393,S,US PRESIDENT,Jill Stein,GRE,1,0,0,0,0,0,Y,NaN
3,CALDWELL,11/05/2024,10,2,C,CALDWELL COUNTY BOARD OF EDUCATION,Michael Barrick,LIB,3,102,0,0,0,102,Y,NaN
4,CALDWELL,11/05/2024,10,4,C,CALDWELL SOIL AND WATER CONSERVATION DISTRICT ...,David Bruner,,1,158,0,0,0,158,Y,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230487,WILKES,11/05/2024,104,1005,S,NC ATTORNEY GENERAL,Jeff Jackson,DEM,1,82,0,0,0,82,Y,NaN
230488,WILKES,11/05/2024,104,1302,S,NC SECRETARY OF STATE,Chad Brown,REP,1,260,0,0,0,260,Y,NaN
230489,WILKES,11/05/2024,104,1338,S,NC STATE SENATE DISTRICT 36,Darren Staley,DEM,1,85,0,0,0,85,Y,NaN
230490,WILKES,11/05/2024,104,1338,S,NC STATE SENATE DISTRICT 36,Eddie Settle,REP,1,270,0,0,0,270,Y,NaN


In [4]:
house_races = unique(df, "Contest Name", "US HOUSE")
house_races

['US HOUSE OF REPRESENTATIVES DISTRICT 07',
 'US HOUSE OF REPRESENTATIVES DISTRICT 06',
 'US HOUSE OF REPRESENTATIVES DISTRICT 08',
 'US HOUSE OF REPRESENTATIVES DISTRICT 09',
 'US HOUSE OF REPRESENTATIVES DISTRICT 14',
 'US HOUSE OF REPRESENTATIVES DISTRICT 11',
 'US HOUSE OF REPRESENTATIVES DISTRICT 03',
 'US HOUSE OF REPRESENTATIVES DISTRICT 04',
 'US HOUSE OF REPRESENTATIVES DISTRICT 01',
 'US HOUSE OF REPRESENTATIVES DISTRICT 10',
 'US HOUSE OF REPRESENTATIVES DISTRICT 05',
 'US HOUSE OF REPRESENTATIVES DISTRICT 13',
 'US HOUSE OF REPRESENTATIVES DISTRICT 12',
 'US HOUSE OF REPRESENTATIVES DISTRICT 02']

In [6]:
overall = None

for race in house_races:
    print("Race: %s", race)
    df_district = filter_for(df, "Contest Name", race)
    district_precincts = df_district.Precinct.unique()

    # presidential votes
    df_pres = filter_isin(df, "Precinct", district_precincts)
    df_pres = filter_for(df_pres, "Contest Name", "US PRESIDENT")

    # governor votes
    df_gov = filter_for(df, "Contest Name", "NC GOVERNOR")
    df_gov = filter_isin(df_gov, "Precinct", district_precincts)

    # Totals for district
    total_house = total_votes_per_precinct(df_district)
    total_pres = total_votes_per_precinct(df_pres)
    total_gov = total_votes_per_precinct(df_gov)

    # Total
    total = pd.merge(total_pres["Total Votes"], total_gov["Total Votes"], on="Precinct")
    total = pd.merge(total, total_house["Total Votes"], on="Precinct")
    total.columns = ["US PRESIDENT", "NC GOVERNOR", "HOUSE"]

    # Calculations
    total["DIFFERENCE"] = total["US PRESIDENT"] - total["HOUSE"]
    total["PERCENTAGE"] = total.DIFFERENCE / total["US PRESIDENT"] * 100

    # Add district
    total["DISTRICT"] = race[-2:]

    # Concatenate
    if overall is None:
        overall = total
        continue

    overall = pd.concat([overall, total])

Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 07
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 06
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 08
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 09
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 14
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 11
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 03
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 04
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 01
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 10
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 05
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 13
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 12
Race: %s US HOUSE OF REPRESENTATIVES DISTRICT 02


In [7]:
overall

,US PRESIDENT,NC GOVERNOR,HOUSE,DIFFERENCE,PERCENTAGE,DISTRICT
Precinct,,,,,,
03,24845,24376,1331,23514,94.642785,07
08,19279,18927,1239,18040,93.573318,07
09,21694,21244,918,20776,95.768415,07
10,14940,14651,667,14273,95.535475,07
11,27317,26694,19,27298,99.930446,07
...,...,...,...,...,...,...
EVWD,10913,10755,9377,1536,14.074956,02
EVWN,19614,19315,16386,3228,16.457632,02
EVZB,10198,10020,3807,6391,62.669151,02


In [8]:
overall.sort_values("PERCENTAGE")[-50:]

,US PRESIDENT,NC GOVERNOR,HOUSE,DIFFERENCE,PERCENTAGE,DISTRICT
Precinct,,,,,,
07,22180,21790,264,21916,98.809739,04
EVWD,10913,10755,124,10789,98.863740,04
EVZB,10198,10020,115,10083,98.872328,04
ABSENTEE,57950,56316,644,57306,98.888697,10
02,28610,27952,317,28293,98.891996,04
03,24845,24376,259,24586,98.957537,04
PROVISIONAL,7548,7275,71,7477,99.059353,10
EVCS,31098,30578,276,30822,99.112483,13
24,11936,11697,96,11840,99.195710,07


In [9]:
overall.sum()

US PRESIDENT                                             14153875
NC GOVERNOR                                              13873930
HOUSE                                                     5482174
DIFFERENCE                                                8671701
PERCENTAGE                                           75630.083036
DISTRICT        0707070707070707070707070707070707070707070707...
dtype: object